In [1]:
import sys
sys.path.append(sys.path[0] + '/bert')
sys.path.append(sys.path[0] + '/flop')
import modeling
import optimization
import tokenization
import run_classifier
import tensorflow as tf
import tensorflow.contrib.slim as slim
import modeling_hardconcrete


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
tf.logging.set_verbosity(tf.logging.DEBUG)
flags = {
    'do_lower_case': True,
    'bert_config_file': '/data0/ultraman/uncased_L-12_H-768_A-12/bert_config.json',
    'init_checkpoint': '/data0/ultraman/uncased_L-12_H-768_A-12/bert_model.ckpt',
    'vocab_file': '/data0/ultraman/uncased_L-12_H-768_A-12/vocab.txt',
    'use_tpu': False,
    'data_dir': '/data0/ultraman/datasets/MNLI',
    'task_name': 'MNLI',
    'learning_rate': 5e-5,
    'is_training': False,
    'train_batch_size': 32,
    'num_train_epochs': 3.0,
    'warmup_proportion': 0.1,
    'predict_batch_size': 8,
    'eval_batch_size': 8,
    'output_dir': './temp_files',
    
}
tokenization.validate_case_matches_checkpoint(flags['do_lower_case'],
flags['init_checkpoint'])
bert_config = modeling.BertConfig.from_json_file(flags['bert_config_file'])
processors = {
      "cola": run_classifier.ColaProcessor,
      "mnli": run_classifier.MnliProcessor,
      "mrpc": run_classifier.MrpcProcessor,
      "xnli": run_classifier.XnliProcessor,
      "mnli": run_classifier.MnliProcessor,
      "qnli": run_classifier.QnliProcessor,
      "qqp": run_classifier.QqpProcessor,
      "rte": run_classifier.RteProcessor,
      "wnli": run_classifier.WnliProcessor,
      "sst-2": run_classifier.Sst2Processor,
      "mrpc": run_classifier.MrpcProcessor,
      "sts-b": run_classifier.StsProcessor,
  }

In [3]:
tokenizer = tokenization.FullTokenizer(vocab_file=flags['vocab_file'], do_lower_case=flags['do_lower_case'])
task_name = flags['task_name'].lower()
processor = processors[task_name]()
label_list = processor.get_labels()
num_train_steps = None
num_warmup_steps = None
input_ids = tf.constant([[31, 51, 99], [15, 5, 0]])
model = modeling_hardconcrete.BertModelHardConcrete(
    config=bert_config,
    is_training=flags['is_training'],
    input_ids=input_ids)




Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [5]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

self/value_p/bias:0 (float32_ref 768) [768, bytes: 3072]
bert/encoder/layer_2/attention/output/dense_q/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert/encoder/layer_2/attention/output/dense_p/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert/encoder/layer_2/attention/output/dense_p/bias:0 (float32_ref 768) [768, bytes: 3072]
bert/encoder/layer_2/attention/output/LayerNorm/beta:0 (float32_ref 768) [768, bytes: 3072]
bert/encoder/layer_2/attention/output/LayerNorm/gamma:0 (float32_ref 768) [768, bytes: 3072]
bert/encoder/layer_2/intermediate/dense_q/kernel:0 (float32_ref 768x768) [589824, bytes: 2359296]
bert/encoder/layer_2/intermediate/dense_p/kernel:0 (float32_ref 768x3072) [2359296, bytes: 9437184]
bert/encoder/layer_2/intermediate/dense_p/bias:0 (float32_ref 3072) [3072, bytes: 12288]
bert/encoder/layer_2/output/dense_q/kernel:0 (float32_ref 3072x768) [2359296, bytes: 9437184]
bert/encoder/layer_2/output/dense_p/kernel:0 (float32_ref 768x768) [589824, bytes: 

In [4]:
sess = tf.Session()
 # 保存路径
tenboard_dir = './tensorboard/fractorized/123'

# 指定一个文件用来保存图
writer = tf.summary.FileWriter(tenboard_dir)
# 把图add进去
writer.add_graph(sess.graph)


In [8]:
import os
from tensorflow.python import pywrap_tensorflow
import re
import copy
import factorize
import numpy as np
checkpoint_path = os.path.join("/data0/ultraman/uncased_L-12_H-768_A-12/", "bert_model.ckpt")
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path) #tf.train.NewCheckpointReader
var_to_shape_map = reader.get_variable_to_shape_map()
kernel_pattern = "^bert/encoder/.*((query|key|value)|(dense))/kernel$"
bias_pattern = "^bert/encoder/.*((query|key|value)|(dense))/bias$"
tvar = tf.trainable_variables()
tvar_names = []
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for var in tvar:
    tvar_names.append(var.name)
for key in var_to_shape_map:
    if re.match(bias_pattern, key):
        q = factorize.bias_map(key)
        q_var = [v for v in tvar if v.name == q][0]
        tf.logging.info("Tensor: %s %s", q, "*INIT_FROM_CKPT*")
        sess.run(tf.assign(q_var, reader.get_tensor(key)))
        tvar_names.remove(q)
    elif re.match(kernel_pattern, key):
        p, q = factorize.kernel_map(key)
        p_var = [v for v in tvar if v.name == p][0]
        q_var = [v for v in tvar if v.name == q][0]
        u, s, v = np.linalg.svd(reader.get_tensor(key))
        smat = np.zeros((u.shape[0], v.shape[0]))
        smat[:s.shape[0], :s.shape[0]] = np.diag(s)
        q_mat = np.dot(smat, v)
        p_mat = u
        tf.logging.info("Tensor: %s %s", p, "*INIT_FROM_CKPT*")
        tf.logging.info("Tensor: %s %s", q, "*INIT_FROM_CKPT*")
        sess.run(tf.assign(p_var, p_mat))
        sess.run(tf.assign(q_var, q_mat))
        tvar_names.remove(p)
        tvar_names.remove(q)
        pass
    elif key + ":0" in tvar_names:
        var = [v for v in tvar if v.name == key + ":0"][0]
        tf.logging.info("Tensor: %s %s", key + ":0", "*INIT_FROM_CKPT*")
        sess.run(tf.assign(var, reader.get_tensor(key)))
        tvar_names.remove(key + ":0")
        pass
    else:
        pass
for var_name in tvar_names:
    tf.logging.info("Tensor: %s %s", var_name, "*NOT_INIT_FROM_CKPT*")
saver = tf.train.Saver()
saver.save(sess,"./checkpoint_dir/bert_f.ckpt")



INFO:tensorflow:Tensor: bert/encoder/layer_3/intermediate/dense_p/kernel:0 *INIT_FROM_CKPT*
INFO:tensorflow:Tensor: bert/encoder/layer_3/intermediate/dense_q/kernel:0 *INIT_FROM_CKPT*
[[-0.02733567  0.03307877 -0.01331289 ... -0.03766872 -0.01103309
  -0.00891848]
 [-0.03209607 -0.08699457  0.00864887 ...  0.02761711  0.03704698
   0.0407665 ]
 [ 0.00527224  0.0083868   0.05706076 ... -0.01937598  0.05206187
   0.0878186 ]
 ...
 [-0.05675254  0.00993388  0.0246828  ...  0.02234653  0.0090737
   0.00302651]
 [ 0.05268594 -0.05868094 -0.02675688 ...  0.01341528  0.01230162
   0.02610132]
 [-0.05411405  0.03780412 -0.01292083 ...  0.01682481  0.0348397
   0.00278786]]
[[-0.02733567  0.03307878 -0.01331292 ... -0.0376687  -0.01103309
  -0.00891848]
 [-0.03209604 -0.08699451  0.00864887 ...  0.0276171   0.03704699
   0.0407665 ]
 [ 0.00527221  0.0083868   0.05706078 ... -0.01937598  0.05206185
   0.08781863]
 ...
 [-0.05675246  0.00993389  0.02468281 ...  0.02234653  0.0090737
   0.00302651

KeyboardInterrupt: 

In [7]:
for var in tf.trainable_variables():
    print(var.name)

bert/embeddings/word_embeddings:0
bert/embeddings/token_type_embeddings:0
bert/embeddings/position_embeddings:0
bert/embeddings/LayerNorm/beta:0
bert/embeddings/LayerNorm/gamma:0
bert/encoder/layer_0/attention/self/query_q/kernel:0
bert/encoder/layer_0/attention/self/query_p/kernel:0
bert/encoder/layer_0/attention/self/query_p/bias:0
bert/encoder/layer_0/attention/self/key_q/kernel:0
bert/encoder/layer_0/attention/self/key_p/kernel:0
bert/encoder/layer_0/attention/self/key_p/bias:0
bert/encoder/layer_0/attention/self/value_q/kernel:0
bert/encoder/layer_0/attention/self/value_p/kernel:0
bert/encoder/layer_0/attention/self/value_p/bias:0
bert/encoder/layer_0/attention/output/dense_q/kernel:0
bert/encoder/layer_0/attention/output/dense_p/kernel:0
bert/encoder/layer_0/attention/output/dense_p/bias:0
bert/encoder/layer_0/attention/output/LayerNorm/beta:0
bert/encoder/layer_0/attention/output/LayerNorm/gamma:0
bert/encoder/layer_0/intermediate/dense_q/kernel:0
bert/encoder/layer_0/intermedia

In [29]:
from tensorflow.python.ops import gen_array_ops
import tensorflow as tf
import numpy as np

data = np.array([[1, 2, 3], [2, 3, 4]])
u, s, v = np.linalg.svd(data)
print(u)
print(np.diag(s))
print(v)
smat = np.zeros((2, 3))
smat[:2, :2] = np.diag(s)
print(smat)
one = np.dot(u, smat)
print(one)
print(np.dot(one, v))

[[-0.56959484 -0.82192562]
 [-0.82192562  0.56959484]]
[[6.54675564 0.        ]
 [0.         0.37415323]]
[[-0.33809817 -0.55064932 -0.76320047]
 [ 0.84795222  0.17354729 -0.50085764]
 [ 0.40824829 -0.81649658  0.40824829]]
[[6.54675564 0.         0.        ]
 [0.         0.37415323 0.        ]]
[[-3.72899821 -0.30752612  0.        ]
 [-5.38094617  0.21311575  0.        ]]
[[1. 2. 3.]
 [2. 3. 4.]]
